In [1]:
import importlib.util
import matplotlib.pyplot as plt
from matplotlib.pyplot import xticks
import numpy as np
import pandas as pd
import os
import time

In [2]:
class Recap():
    
    def __init__(self):
        # Filepath to Excel raw file
        #self.origin_file = "./raw/Activities_B_test.xlsx"
        self.origin_file = "./raw/Activities_B.xlsx"
        
        # Filepath to Excel treated file
        self.target_file = "./treated/Activities_B.xlsx"
        
        # Start_time
        self.start_time = "13:05:00"
        
        # Raw Data
        self.raw_data = None
        self.get_raw_data()
        
        # Treated Data
        self.treated_data = None
        
        # All submissions list
        self.all_subs_list = []
        self.get_all_subs()
        
        # Grade submissions
        self.grades_list = []
        self.get_all_grades()
        
        # Save to file
        self.recreate_pd_data()
        self.save_to_excel()
        
        
    def get_raw_data(self):
        
        # Read raw data from file
        self.raw_data = pd.read_excel(self.origin_file).astype(str).to_numpy()
        
    def get_all_subs(self):
        
        for submission in self.raw_data:
            self.all_subs_list.append(Submission(submission))
            
    def get_all_grades(self):
        
        for submission in self.all_subs_list:
            self.grades_list.append((submission.name, submission.ID, submission.total_grade))
        
        self.unique_grades_list = []
        for grade in self.grades_list:
            ID_list = [tup[1] for tup in self.unique_grades_list]
            if not grade[1] in ID_list:
                self.unique_grades_list.append(grade)
            else:
                print("Duplicate found for: ", grade[1])
                
    def recreate_pd_data(self):
        
        # Initialize dictionary
        data_dic = {"Name": [], "ID": [], "Time to submit": []}
        data_dic.update({"Q{}".format(i+1): [] for i in range(5)})
        data_dic.update({"Total Activities score": []})
        
        # Add entries
        for submission in self.all_subs_list:
            data_dic["Name"].append(submission.name)
            data_dic["ID"].append(submission.ID)
            data_dic["Time to submit"].append(submission.elapsed_time)
            for index, score in enumerate(submission.scores_list):
                data_dic["Q{}".format(index + 1)].append(score)
            data_dic["Total Activities score"].append(submission.total_grade)
        
        # To pandas
        self.treated_data = pd.DataFrame(data_dic)
        
    
    def save_to_excel(self):
        
        self.treated_data.to_excel(self.target_file, engine = "xlsxwriter") 

In [3]:
class Question():
    
    def __init__(self, number, points, f_as_str):
        # Question number
        self.number = number
        
        # Number of points
        self.points = points
        
        # Function as string
        self.f_as_str = f_as_str
        
        
    def str_to_file(self, filepath, filename):
        # Send file in string format to file
        path_to_file = filepath + filename
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        with open(path_to_file, "w") as file:
            file.write(self.f_as_str)
            file.close()
        self.path_to_file = path_to_file

In [4]:
class Question1(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 1, points = 10, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("convert_seconds", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "convert_seconds")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                weeks, days, hours = self.function_name(8)
                total_points += points_per_tc*int(weeks == 0 and days == 0 and hours == 8)
                weeks, days, hours = self.function_name(69)
                total_points += points_per_tc*int(weeks == 0 and days == 1 and hours == 9)
                weeks, days, hours = self.function_name(3618)
                total_points += points_per_tc*int(weeks == 1 and days == 0 and hours == 18)
                weeks, days, hours = self.function_name(7272)
                total_points += points_per_tc*int(weeks == 2 and days == 1 and hours == 12)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [5]:
class Question2(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 2, points = 10, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("old_enough", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "old_enough")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res1, res2 = self.function_name(22)
                total_points += points_per_tc*int(res1 == True and res2 is None)
                res1, res2 = self.function_name(21)
                total_points += points_per_tc*int(res1 == True and res2 is None)
                res1, res2 = self.function_name(19)
                total_points += points_per_tc*int(res1 == False and res2 == 2)
                res1, res2 = self.function_name(43)
                total_points += points_per_tc*int(res1 == True and res2 is None)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [6]:
class Question3(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 3, points = 20, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("how_many_months", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "how_many_months")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res = self.function_name(20000, 1.0)
                total_points += points_per_tc*int(res == 394)
                res = self.function_name(100000, 2.0)
                total_points += points_per_tc*int(res == 117)
                res = self.function_name(1000000, 1.0)
                total_points += points_per_tc*int(res == 0)
                res = self.function_name(20000, 10.0)
                total_points += points_per_tc*int(res == 42)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [7]:
class Question4(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 4, points = 30, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("are_all_consecutive", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "are_all_consecutive")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res = self.function_name([9])
                total_points += points_per_tc*int(res == True)
                res = self.function_name([4,5,6,7])
                total_points += points_per_tc*int(res == True)
                res = self.function_name([3,3,4,5,6])
                total_points += points_per_tc*int(res == False)
                res = self.function_name([6,7,9,10])
                total_points += points_per_tc*int(res == False)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [8]:
class Question5(Question):
    
    def __init__(self, f_as_str):
        # Reusing mother class init
        super().__init__(number = 5, points = 30, f_as_str = f_as_str)
        
        
    def import_from_file(self, filepath, filename):
        try:
            # Import function from file
            self.str_to_file(filepath, filename)
            spec = importlib.util.spec_from_file_location("is_prime", self.path_to_file)
            function = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(function)
            self.function_name = getattr(function, "is_prime")
            self.malfunctioning = False
        except:
            self.malfunctioning = True
        
    
    def grade(self):
        # Grade function by test cases
        test_cases_number = 4
        total_points = 0
        points_per_tc = self.points/test_cases_number
        print("-----")
        if(self.malfunctioning):
            self.override = True
            print("Could not import function. Displaying function below. \n")
            print(self.f_as_str, end = "\n\n")
            total_points = -1
            while(total_points < 0 or total_points > self.points):
                time.sleep(1)
                total_points = float(input("How many points? (max. {}): ".format(self.points)))
            return total_points
        else:
            try:
                res = self.function_name(7)
                total_points += points_per_tc*int(res == True)
                res = self.function_name(62)
                total_points += points_per_tc*int(res == False)
                res = self.function_name(131)
                total_points += points_per_tc*int(res == True)
                res = self.function_name(1)
                total_points += points_per_tc*int(res == False)
                self.override = False
                print("Testing function below. \n")
                print(self.f_as_str, end = "\n\n")
                print("Function was given {} / {} by autograder".format(total_points, self.points))
                assert total_points == self.points, "Did not obtain maximal score."
                return total_points
            except:
                self.override = True
                print("Error detected in function. Displaying function below. \n")
                print(self.f_as_str, end = "\n\n")
                total_points = -1
                while(total_points < 0 or total_points > self.points):
                    time.sleep(1)
                    total_points = float(input("How many points? (max. {}): ".format(self.points)))
                return total_points

In [9]:
class Submission():
    
    def __init__(self, student_sub):
        # Student name
        self.name = None
        self.get_name(student_sub)
        
        # Student ID
        self.ID = None
        self.get_ID(student_sub)
        
        # Timestamp
        self.timestamp = None
        self.convert_timestamp(student_sub)
        
        # Elapsed time
        self.elapsed_time = None
        
        # Answers_list
        self.answers_list = []
        print("--------------------")
        print("Checking submission for student {}, with ID {}.".format(self.name, self.ID))
        self.get_answers_list(student_sub)
        
        # Scores list
        self.scores_list = []
        
        # Total grade
        self.total_grade = 0
        self.grade_questions()
        
        
    def get_name(self, student_sub):
        self.name = student_sub[1]
        
        
    def get_ID(self, student_sub):
        self.ID = student_sub[2]
        
        
    def convert_timestamp(self, student_sub):
        self.timestamp = student_sub[0][-12:-4]
        
        
    def get_answers_list(self, student_sub):
        # Recreate questions objects with student submissions
        self.answers_list = []
        filepath = "./functions/" + self.ID + "/"
        # Q1
        q1 = Question1(student_sub[3])
        q1.import_from_file(filepath, "convert_hours.py")
        self.answers_list.append(q1)
        # Q2
        q2 = Question2(student_sub[4])
        q2.import_from_file(filepath, "is_leap_year.py")
        self.answers_list.append(q2)
        # Q3
        q3 = Question3(student_sub[5])
        q3.import_from_file(filepath, "how_many_divide.py")
        self.answers_list.append(q3)
        # Q4
        q4 = Question4(student_sub[6])
        q4.import_from_file(filepath, "find_if_triplet.py")
        self.answers_list.append(q4)
        # Q5
        q5 = Question5(student_sub[7])
        q5.import_from_file(filepath, "is_sum_squares.py")
        self.answers_list.append(q5)
        
        
    def grade_questions(self):
        # Grade submissions and compute score for this submission
        for question in self.answers_list:
            self.scores_list.append(question.grade())
        self.total_grade += self.scores_list[0]
        self.total_grade += self.scores_list[1]
        self.total_grade += self.scores_list[2]
        self.total_grade += self.scores_list[3]
        self.total_grade += self.scores_list[4]

In [10]:
r = Recap()
print(r.__dict__.keys())

--------------------
Checking submission for student Ng Minzheng Joel, with ID 1005032.
(0, 0, 7)
(0, 1, 7)
(1, 0, 25)
(True, None)
(False, 3)
(True, None)
463
95
48
0
True
True
True
False
False
False
True
False
-----
Testing function below. 

def convert_seconds(total_seconds):
    hours = 0
    minutes = 0
    if total_seconds >= 3600:
        hours = total_seconds//3600
        total_seconds -= hours*3600
        
    if total_seconds >= 60:
        minutes = total_seconds//60
        total_seconds -= minutes*60
    
    seconds = total_seconds
    
    return hours, minutes, seconds

print(convert_seconds(7))
print(convert_seconds(67))
print(convert_seconds(3625))

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def old_enough(age):
    number = None
    if age >= 21:
        can_vote = True
    else:
        can_vote = False
        number = 21 - age
    
    return can_vote, number

print(old_enough(42))
print(old_enough(18))
print(old_enough(21))

Func

How many points? (max. 30): 25
--------------------
Checking submission for student Ernest Ng Wei Jun, with ID 1005212.
(0, 0, 7)
(0, 1, 7)
(1, 0, 25)
(True, None)
(False, 3)
(True, None)
463
95
48
0
True
True
True
False
False
False
True
False
-----
Testing function below. 

def convert_seconds(total_seconds):
    hours = total_seconds // 3600
    total_seconds -= hours*3600
    minutes = total_seconds // 60
    total_seconds -= minutes*60
    seconds = total_seconds 
    return hours, minutes, seconds

print(convert_seconds(7))
print(convert_seconds(67))
print(convert_seconds(3625))

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def old_enough(age):
    can_vote = None
    number = None
    if age >= 21:
        can_vote = True
        number = None
    elif age < 21:
        can_vote = False
        number = 21-age
    return can_vote, number

print(old_enough(42))
print(old_enough(18))
print(old_enough(21))

Function was given 10.0 / 10 by autograder
---

How many points? (max. 30): 10
-----
Testing function below. 

def is_prime(number):
    
    if(number == 1):
        is_p = False
        
    elif(number%number == 0 and number%1 == 0):
        counter = 1
        while(counter < number - 1):
            counter += 1
            if(number%counter == 0):
                is_p = False
                break
            elif(not number%counter == 0 and counter == number - 1):
                is_p = True
        
    return is_p

Function was given 30.0 / 30 by autograder
--------------------
Checking submission for student Ian Ng Zhi Yang, with ID 1005640.
(0, 0, 7)
(0, 1, 7)
(0, 17, 4)
(0, 2, 3)
(True, None)
(False, 3)
(True, None)
-----
Testing function below. 

def convert_seconds(total_seconds):
    
    seconds = total_seconds % 60
    
    hours = total_seconds // (60**2)
    minutes = int((total_seconds / (60**2) - hours ) * 60 )
    
    return hours, minutes, seconds

print(convert_seconds(7))
print(convert_seconds(67))
print(co

How many points? (max. 30): 25
--------------------
Checking submission for student Christy Lau Jin Yun, with ID 1005330.
-----
Testing function below. 

def convert_seconds(total_seconds):
    hours = total_seconds // 3600
    minutes = (total_seconds - hours*3600) // 60
    seconds = total_seconds - minutes*60 - hours*3600
    return hours, minutes, seconds

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def old_enough(age):
    if(age >= 21):
        can_vote = True
    else:
        can_vote = False
        
    if(can_vote == False):
        number = 21 - age
    else:
        number = None
    return can_vote, number

Function was given 10.0 / 10 by autograder
-----
Testing function below. 

def how_many_months(initial_amount, rate):
    months = 0
    while initial_amount < 1000000:
        initial_amount *= 1 + rate/100
        months += 1
    return months

Function was given 20.0 / 20 by autograder
-----
Testing function below. 

def are_all_consec

How many points? (max. 30): 0
-----
Could not import function. Displaying function below. 

def prime_number():
    number = int(input('Number'))
    if number<0:
        print("Your number must either be zero or ppositive")
        y = 1
        while(number%y ==0):
            y = y+1
            if(number%y>0 and number - y >0):
                print("Your number is not a prime number")
            else:
                print("Your number is a prime number")

How many points? (max. 30): 0
--------------------
Checking submission for student Christopher Wah Boon Chuan, with ID 1005214.
-----
Testing function below. 

def convert_seconds(total_seconds):
    hours = 0
    minutes = 0 
    seconds = 0
    while total_seconds > 0:
        while total_seconds - 3600 >= 0:
            hours += 1
            total_seconds -= 3600
        while total_seconds - 60 >= 0:
            minutes += 1
            total_seconds -= 60
        seconds = total_seconds
        total_seconds -= seconds
  

...

How many points? (max. 20): 10
-----
Testing function below. 

def are_all_consecutive(my_list):
    if(my_list == my_list.sort()):
            is_cons = True
            
    else:
        is_cons = False
        
    return is_cons

Function was given 15.0 / 30 by autograder
Error detected in function. Displaying function below. 

def are_all_consecutive(my_list):
    if(my_list == my_list.sort()):
            is_cons = True
            
    else:
        is_cons = False
        
    return is_cons

How many points? (max. 30): 0
-----
Testing function below. 

def is_prime(number):
    if(number == 1): 
        return False
    elif(number == 2):
        return True
    else:
        for i in range(2,number):
            if(number % i==0):
                return False

Function was given 15.0 / 30 by autograder
Error detected in function. Displaying function below. 

def is_prime(number):
    if(number == 1): 
        return False
    elif(number == 2):
        return True
    else:


How many points? (max. 30): 25
-----
Testing function below. 

def is_prime(number):
    n=3
    is_p = None
    if number <= 1:
        is_p = False
    elif number == 2:
        is_p = True
    elif number == 3:
        is_p = True
    elif number%2 == 0:
        is_p = False
    else:
        while number >= n:
            if number == n:
                is_p = True
                break
            elif (number%n == 0):
                is_p = False
                break
            elif(number%n != 0):
                n += 1
               
    return is_p

Function was given 30.0 / 30 by autograder
--------------------
Checking submission for student Kymbelin Woong, with ID 1005236.
(0, 0, 7)
(0, 1, 7)
(1, 0, 25)
(True, None)
(False, 3)
(True, None)
463
95
48
0
True
True
True
True
True
False
True
False
-----
Testing function below. 

def convert_seconds(total_seconds):
    if total_seconds<60:
        hours=0
        minutes=0
        seconds= total_seconds
        return  hours, 

amount so far: 77615.87555117437
months so far: 42
amount so far: 81496.66932873309
months so far: 43
amount so far: 85571.50279516974
months so far: 44
amount so far: 89850.07793492824
months so far: 45
amount so far: 94342.58183167466
months so far: 46
amount so far: 99059.71092325839
months so far: 47
amount so far: 104012.69646942131
months so far: 48
amount so far: 109213.33129289238
months so far: 49
amount so far: 114673.997857537
months so far: 50
amount so far: 120407.69775041386
months so far: 51
amount so far: 126428.08263793457
months so far: 52
amount so far: 132749.4867698313
months so far: 53
amount so far: 139386.96110832287
months so far: 54
amount so far: 146356.309163739
months so far: 55
amount so far: 153674.12462192596
months so far: 56
amount so far: 161357.83085302226
months so far: 57
amount so far: 169425.72239567337
months so far: 58
amount so far: 177897.00851545704
months so far: 59
amount so far: 186791.8589412299
months so far: 60
amount so far: 196131.45

months so far: 5
amount so far: 35431.22000000002
months so far: 6
amount so far: 38974.342000000026
months so far: 7
amount so far: 42871.77620000003
months so far: 8
amount so far: 47158.95382000004
months so far: 9
amount so far: 51874.84920200005
months so far: 10
amount so far: 57062.33412220006
months so far: 11
amount so far: 62768.56753442007
months so far: 12
amount so far: 69045.42428786209
months so far: 13
amount so far: 75949.9667166483
months so far: 14
amount so far: 83544.96338831313
months so far: 15
amount so far: 91899.45972714445
months so far: 16
amount so far: 101089.40569985891
months so far: 17
amount so far: 111198.34626984481
months so far: 18
amount so far: 122318.1808968293
months so far: 19
amount so far: 134549.99898651225
months so far: 20
amount so far: 148004.99888516348
months so far: 21
amount so far: 162805.49877367984
months so far: 22
amount so far: 179086.04865104784
months so far: 23
amount so far: 196994.65351615264
months so far: 24
amount so f

How many points? (max. 30): 20
-----
Testing function below. 

def is_prime(number):
    divisors = []
    
    for value in range(1, number+1):  
        if(number % value == 0):
            divisors.append(value)
    
    if len(divisors) == 2:
        is_p=True
        
    else:
        is_p=False
        
    return is_p
    
print(is_prime(1))
print(is_prime(3))
print(is_prime(6))

Function was given 30.0 / 30 by autograder
--------------------
Checking submission for student Liew Soon Hao, with ID 1005003.
(0, 0, 7)
(0, 1, 7)
(0, 17, 4)
(1, 0, 25)
462.8157851175219
94.38726563812871
47.193632819064355
0.0
-----
Testing function below. 

def convert_seconds(total_seconds):
    total_seconds>0
    hours = total_seconds//3600
    total_seconds%= 3600
    minutes = total_seconds//60
    total_seconds%=60
    seconds = total_seconds%60
    return hours, minutes, seconds

print(convert_seconds(7))
print(convert_seconds(67))
print(convert_seconds(1024))
print (convert_seconds(3625))

F

In [11]:
print(len(r.grades_list))

78


In [12]:
print(r.grades_list)

[('Ng Minzheng Joel', '1005032', 100.0), ('Luo Yehao Benjamin', '1005368', 100.0), ('Yap Zhan Hao, Sean', '1005153', 100.0), ('Leong Keng Hoy', '1005164', 100.0), ('Tay Wei Han Joel', '1005117', 95.0), ('Ernest Ng Wei Jun', '1005212', 100.0), ('Pala Tej Deep', '1005282', 100.0), ('Gabriel Teng', '1005027', 90.0), ('ANN MARY ALEN', '1005382', 100.0), ('Theresa Lam', '1005477', 80.0), ('Ian Ng Zhi Yang', '1005640', 100.0), ('Adelle Chan Min Hui', '1005418', 100.0), ('Yee Li Ren, Harel', '1005133', 100.0), ('Pung Junhao Louis', '1005039', 100.0), ('Yong Zhe Rui Gabriel', '1005154', 95.0), ('Christy Lau Jin Yun', '1005330', 100.0), ('Ryan Pan Tang Kai', '1005037', 100.0), ('Ankita Parashar', '1005478', 16.0), ('Josiah Fong', '1005465', 0.0), ('Christopher Wah Boon Chuan', '1005214', 100.0), ('Lee Eharn Natthan', '1005006', 100.0), ('Emmet Wee', '1005475', 95.0), ('Michael Chun Kai Peng', '1005258', 100.0), ('Hnin Myat Eindray Oo', '1005315', 10.0), ('Pyae Phyo Aung @ Elvis Kason Lin', '100

In [13]:
print(len(r.unique_grades_list))

72


In [14]:
print(r.unique_grades_list)

[('Ng Minzheng Joel', '1005032', 100.0), ('Luo Yehao Benjamin', '1005368', 100.0), ('Yap Zhan Hao, Sean', '1005153', 100.0), ('Leong Keng Hoy', '1005164', 100.0), ('Tay Wei Han Joel', '1005117', 95.0), ('Ernest Ng Wei Jun', '1005212', 100.0), ('Pala Tej Deep', '1005282', 100.0), ('Gabriel Teng', '1005027', 90.0), ('ANN MARY ALEN', '1005382', 100.0), ('Theresa Lam', '1005477', 80.0), ('Ian Ng Zhi Yang', '1005640', 100.0), ('Adelle Chan Min Hui', '1005418', 100.0), ('Yee Li Ren, Harel', '1005133', 100.0), ('Pung Junhao Louis', '1005039', 100.0), ('Yong Zhe Rui Gabriel', '1005154', 95.0), ('Christy Lau Jin Yun', '1005330', 100.0), ('Ryan Pan Tang Kai', '1005037', 100.0), ('Ankita Parashar', '1005478', 16.0), ('Josiah Fong', '1005465', 0.0), ('Christopher Wah Boon Chuan', '1005214', 100.0), ('Lee Eharn Natthan', '1005006', 100.0), ('Emmet Wee', '1005475', 95.0), ('Michael Chun Kai Peng', '1005258', 100.0), ('Hnin Myat Eindray Oo', '1005315', 10.0), ('Pyae Phyo Aung @ Elvis Kason Lin', '100

In [15]:
# To be executed to store custom grades to files
import pickle
save = True
if(save):
    with open("./pickle_B.pkl", "wb") as f:
        pickle.dump(r.unique_grades_list, f)
        f.close()